### (1) read training file: extract (image, label) tuples

In [1]:
with open('../../../data/isbi2016_isic_part1_train_v8.list', 'r') as training_file:
    images_labels_train = training_file.readlines()
    
images_train=[]
labels_train=[]
for image_label in images_labels_train:
    i_l = image_label.split()
    images_train.append(i_l[0])
    labels_train.append(i_l[1])
    
print(dict(zip(images_train,labels_train)))

{'ISIC_0002780.jpg': 'ISIC_0002780_Segmentation.png', 'ISIC_0002780_01.jpg': 'ISIC_0002780_Segmentation_01.png', 'ISIC_0009860.jpg': 'ISIC_0009860_Segmentation.png', 'ISIC_0009860_01.jpg': 'ISIC_0009860_Segmentation_01.png', 'ISIC_0000091.jpg': 'ISIC_0000091_Segmentation.png', 'ISIC_0000091_01.jpg': 'ISIC_0000091_Segmentation_01.png', 'ISIC_0010351.jpg': 'ISIC_0010351_Segmentation.png', 'ISIC_0010351_01.jpg': 'ISIC_0010351_Segmentation_01.png', 'ISIC_0010593.jpg': 'ISIC_0010593_Segmentation.png', 'ISIC_0010593_01.jpg': 'ISIC_0010593_Segmentation_01.png', 'ISIC_0010003.jpg': 'ISIC_0010003_Segmentation.png', 'ISIC_0010003_01.jpg': 'ISIC_0010003_Segmentation_01.png', 'ISIC_0010435.jpg': 'ISIC_0010435_Segmentation.png', 'ISIC_0010435_01.jpg': 'ISIC_0010435_Segmentation_01.png', 'ISIC_0000249.jpg': 'ISIC_0000249_Segmentation.png', 'ISIC_0000249_01.jpg': 'ISIC_0000249_Segmentation_01.png', 'ISIC_0000124.jpg': 'ISIC_0000124_Segmentation.png', 'ISIC_0000124_01.jpg': 'ISIC_0000124_Segmentation_

In [2]:
len(images_train)

1399

### (2) read validation file: extract (image, label) tuples

In [3]:
with open('../../../data/isbi2016_isic_part1_val_v8.list', 'r') as validation_file:
    images_labels_val = validation_file.readlines()

images_val=[]
labels_val=[]
for image_label in images_labels_val:
    i_l = image_label.split()
    images_val.append(i_l[0])
    labels_val.append(i_l[1])
    
print(dict(zip(images_val,labels_val)))

{'ISIC_0000423.jpg': 'ISIC_0000423_Segmentation.png', 'ISIC_0002093.jpg': 'ISIC_0002093_Segmentation.png', 'ISIC_0009877.jpg': 'ISIC_0009877_Segmentation.png', 'ISIC_0010225.jpg': 'ISIC_0010225_Segmentation.png', 'ISIC_0001188.jpg': 'ISIC_0001188_Segmentation.png', 'ISIC_0000385.jpg': 'ISIC_0000385_Segmentation.png', 'ISIC_0009960.jpg': 'ISIC_0009960_Segmentation.png', 'ISIC_0003174.jpg': 'ISIC_0003174_Segmentation.png', 'ISIC_0000301.jpg': 'ISIC_0000301_Segmentation.png', 'ISIC_0010380.jpg': 'ISIC_0010380_Segmentation.png', 'ISIC_0000281.jpg': 'ISIC_0000281_Segmentation.png', 'ISIC_0010358.jpg': 'ISIC_0010358_Segmentation.png', 'ISIC_0000031.jpg': 'ISIC_0000031_Segmentation.png', 'ISIC_0010558.jpg': 'ISIC_0010558_Segmentation.png', 'ISIC_0011380.jpg': 'ISIC_0011380_Segmentation.png', 'ISIC_0006800.jpg': 'ISIC_0006800_Segmentation.png', 'ISIC_0010191.jpg': 'ISIC_0010191_Segmentation.png', 'ISIC_0000370.jpg': 'ISIC_0000370_Segmentation.png', 'ISIC_0000452.jpg': 'ISIC_0000452_Segmentatio

### (3) read test file: extract (image, label) tuples

In [4]:
len(images_val)

90

In [5]:
with open('../../../data/isbi2016_isic_part1_test_v8.list', 'r') as test_file:
    images_labels_test = test_file.readlines()

images_test=[]
labels_test=[]
for image_label in images_labels_test:
    i_l = image_label.split()
    images_test.append(i_l[0])
    labels_test.append(i_l[1])
    
print(dict(zip(images_test,labels_test)))

{'ISIC_0000003.jpg': 'ISIC_0000003_Segmentation.png', 'ISIC_0000012.jpg': 'ISIC_0000012_Segmentation.png', 'ISIC_0000013.jpg': 'ISIC_0000013_Segmentation.png', 'ISIC_0000014.jpg': 'ISIC_0000014_Segmentation.png', 'ISIC_0000015.jpg': 'ISIC_0000015_Segmentation.png', 'ISIC_0000020.jpg': 'ISIC_0000020_Segmentation.png', 'ISIC_0000022.jpg': 'ISIC_0000022_Segmentation.png', 'ISIC_0000023.jpg': 'ISIC_0000023_Segmentation.png', 'ISIC_0000027.jpg': 'ISIC_0000027_Segmentation.png', 'ISIC_0000036.jpg': 'ISIC_0000036_Segmentation.png', 'ISIC_0000037.jpg': 'ISIC_0000037_Segmentation.png', 'ISIC_0000040.jpg': 'ISIC_0000040_Segmentation.png', 'ISIC_0000043.jpg': 'ISIC_0000043_Segmentation.png', 'ISIC_0000052.jpg': 'ISIC_0000052_Segmentation.png', 'ISIC_0000053.jpg': 'ISIC_0000053_Segmentation.png', 'ISIC_0000056.jpg': 'ISIC_0000056_Segmentation.png', 'ISIC_0000057.jpg': 'ISIC_0000057_Segmentation.png', 'ISIC_0000064.jpg': 'ISIC_0000064_Segmentation.png', 'ISIC_0000066.jpg': 'ISIC_0000066_Segmentatio

### (4) compute per-channel mean and build tfrecords from training images

In [6]:
import numpy as np
from PIL import Image
import tensorflow as tf
import os
import io
from PIL import ImageStat

train_data_dir = '../../../data/ISBI2016_ISIC_Part1_Training_Data_v8_processed'

mean_rgb = np.array([0., 0., 0.])
count=0

def _bytes_feature(value): 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

writer = tf.python_io.TFRecordWriter('melanoma_train_v8.tfrecords')

cnt=0

for fimg, flabel in zip(images_train,labels_train):
    #print(fimg, flabel)
    bytes_io_img = io.BytesIO()
    img = Image.open(os.path.join(train_data_dir,fimg))
    stat = ImageStat.Stat(img)
    mean_rgb = mean_rgb + np.array(stat.mean)
    count += 1
    img.save(bytes_io_img, 'PNG')
    bytes_io_lbl = io.BytesIO()
    label = Image.open(os.path.join(train_data_dir,flabel))
    label.save(bytes_io_lbl, 'PNG')
    example = tf.train.Example(
        features=tf.train.Features(
            feature={'image': _bytes_feature(bytes_io_img.getvalue()),
                     'image_name': _bytes_feature(str.encode(fimg)),
                     'label_name': _bytes_feature(str.encode(flabel)),
                     'label': _bytes_feature(bytes_io_lbl.getvalue())
                    }
        )
    )
    writer.write(example.SerializeToString())
    cnt+=1
    if cnt % 50 == 0:
        print('completed percentage %{}'.format(100*cnt/(len(images_train))))
writer.close()

print('completed percentage %{}'.format(100*cnt/(len(images_train))))

np.save('mean_rgb_train_v8.npy', mean_rgb/count)

completed percentage %3.5739814152966405
completed percentage %7.147962830593281
completed percentage %10.72194424588992
completed percentage %14.295925661186562
completed percentage %17.8699070764832
completed percentage %21.44388849177984
completed percentage %25.017869907076484
completed percentage %28.591851322373124
completed percentage %32.16583273766977
completed percentage %35.7398141529664
completed percentage %39.313795568263046
completed percentage %42.88777698355968
completed percentage %46.461758398856325
completed percentage %50.03573981415297
completed percentage %53.609721229449605
completed percentage %57.18370264474625
completed percentage %60.75768406004289
completed percentage %64.33166547533953
completed percentage %67.90564689063616
completed percentage %71.4796283059328
completed percentage %75.05360972122945
completed percentage %78.62759113652609
completed percentage %82.20157255182274
completed percentage %85.77555396711936
completed percentage %89.34953538241

### (5) build tfrecords from validation images

In [7]:
writer = tf.python_io.TFRecordWriter('melanoma_val_v8.tfrecords')

for fimg, flabel in zip(images_val,labels_val):
    print(fimg, flabel)
    bytes_io_img = io.BytesIO()
    img = Image.open(os.path.join(train_data_dir,fimg))
    img.save(bytes_io_img, 'PNG')
    bytes_io_lbl = io.BytesIO()
    label = Image.open(os.path.join(train_data_dir,flabel))
    label.save(bytes_io_lbl, 'PNG')
    example = tf.train.Example(
        features=tf.train.Features(
            feature={'image': _bytes_feature(bytes_io_img.getvalue()),
                     'image_name': _bytes_feature(str.encode(fimg)),
                     'label_name': _bytes_feature(str.encode(flabel)),
                     'label': _bytes_feature(bytes_io_lbl.getvalue())
                    }
        )
    )
    writer.write(example.SerializeToString())
writer.close()

ISIC_0000423.jpg ISIC_0000423_Segmentation.png
ISIC_0002093.jpg ISIC_0002093_Segmentation.png
ISIC_0009877.jpg ISIC_0009877_Segmentation.png
ISIC_0010225.jpg ISIC_0010225_Segmentation.png
ISIC_0001188.jpg ISIC_0001188_Segmentation.png
ISIC_0000385.jpg ISIC_0000385_Segmentation.png
ISIC_0009960.jpg ISIC_0009960_Segmentation.png
ISIC_0003174.jpg ISIC_0003174_Segmentation.png
ISIC_0000301.jpg ISIC_0000301_Segmentation.png
ISIC_0010380.jpg ISIC_0010380_Segmentation.png
ISIC_0000281.jpg ISIC_0000281_Segmentation.png
ISIC_0010358.jpg ISIC_0010358_Segmentation.png
ISIC_0000031.jpg ISIC_0000031_Segmentation.png
ISIC_0010558.jpg ISIC_0010558_Segmentation.png
ISIC_0011380.jpg ISIC_0011380_Segmentation.png
ISIC_0006800.jpg ISIC_0006800_Segmentation.png
ISIC_0010191.jpg ISIC_0010191_Segmentation.png
ISIC_0000370.jpg ISIC_0000370_Segmentation.png
ISIC_0000452.jpg ISIC_0000452_Segmentation.png
ISIC_0000218.jpg ISIC_0000218_Segmentation.png
ISIC_0011297.jpg ISIC_0011297_Segmentation.png
ISIC_0011315.

### (6) build tfrecords from test images

In [8]:
test_data_dir = os.path.normpath('C:/Users/kirta/Documents/research/data/ISBI2016_ISIC_Part1_Test_Data')
test_label_dir = os.path.normpath('C:/Users/kirta/Documents/research/data\ISBI2016_ISIC_Part1_Test_GroundTruth')

writer = tf.python_io.TFRecordWriter('melanoma_test_v8.tfrecords')

cnt=0
for fimg, flabel in zip(images_test, labels_test):
    #print(fimg, flabel)
    bytes_io_img = io.BytesIO()
    img = Image.open(os.path.join(test_data_dir,fimg))
    img.save(bytes_io_img, 'PNG')
    bytes_io_lbl = io.BytesIO()
    label = Image.open(os.path.join(test_label_dir,flabel))
    label.save(bytes_io_lbl, 'PNG')
    example = tf.train.Example(
        features=tf.train.Features(
            feature={'image': _bytes_feature(bytes_io_img.getvalue()),
                     'image_name': _bytes_feature(str.encode(fimg)),
                     'label_name': _bytes_feature(str.encode(flabel)),
                     'label': _bytes_feature(bytes_io_lbl.getvalue())
                    }
        )
    )
    writer.write(example.SerializeToString())
    cnt+=1
    if cnt % 50 == 0:
        print('completed percentage %{}'.format(100*cnt/(len(images_test))))
writer.close()
print('completed percentage %{}'.format(100*cnt/(len(images_test))))

completed percentage %13.192612137203167
completed percentage %26.385224274406333
completed percentage %39.577836411609496
completed percentage %52.770448548812666
completed percentage %65.96306068601584
completed percentage %79.15567282321899
completed percentage %92.34828496042216
completed percentage %100.0
